In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"
# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,051 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,346 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Ge

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("SELECT date_built, ROUND(AVG(price),2) AS average_price FROM home_sales WHERE bedrooms==4 GROUP BY date_built").show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2016|    296050.24|
|      2012|    298233.42|
|      2017|    296576.69|
|      2014|    299073.89|
|      2013|    299999.39|
|      2011|     302141.9|
|      2015|    307908.86|
|      2010|    296800.75|
+----------+-------------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("SELECT date_built, ROUND(AVG(price),2) AS average_price FROM home_sales WHERE bedrooms==3 AND bathrooms==3 GROUP BY date_built").show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2016|    290555.07|
|      2012|    293683.19|
|      2017|    292676.79|
|      2014|    290852.27|
|      2013|    295962.27|
|      2011|    291117.47|
|      2015|     288770.3|
|      2010|    292859.62|
+----------+-------------+



In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("SELECT date_built, ROUND(AVG(price),2) AS average_price FROM home_sales \
WHERE bedrooms==3 AND \
bathrooms==3 AND \
floors==2 AND \
sqft_lot>=2000 \
GROUP BY date_built").show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2016|     287812.6|
|      2012|    295858.68|
|      2017|    282026.59|
|      2014|    294195.13|
|      2013|    295142.13|
|      2011|    281413.45|
|      2015|    291788.36|
|      2010|    280447.23|
+----------+-------------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time = time.time()

spark.sql("SELECT ROUND(AVG(view),2) AS average_view \
FROM home_sales \
WHERE price >= 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))

+------------+
|average_view|
+------------+
|       32.26|
+------------+

--- 0.8315849304199219 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
start_time = time.time()

spark.sql("SELECT ROUND(AVG(view),2) AS average_view \
FROM home_sales \
WHERE price >= 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))

+------------+
|average_view|
+------------+
|       32.26|
+------------+

--- 0.6007461547851562 seconds ---


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned") 

In [13]:
# 11. Read the parquet formatted data.
p=spark.read.parquet("home_sales_partitioned")

In [14]:
# 12. Create a temporary table for the parquet data.
p.createOrReplaceTempView("parquetted_home_sales")

In [15]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("SELECT ROUND(AVG(view),2) AS average_view \
FROM home_sales \
WHERE price >= 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))

+------------+
|average_view|
+------------+
|       32.26|
+------------+

--- 0.5398697853088379 seconds ---


In [16]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [17]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False